# <font color='blue'>Data Science Academy - Python Fundamentos - Capítulo 6</font>

## Download: http://github.com/dsacademybr

## Stream de Dados do Twitter com MongoDB, Pandas e Scikit Learn

## Preparando a Conexão com o Twitter

In [1]:
# Instala o pacote tweepy
!pip install tweepy

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
# Importando os módulos Tweepy, Datetime e Json
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime
import json

Veja no manual em pdf como criar sua API no Twitter e configure as suas chaves abaixo.

In [3]:
# Adicione aqui sua Consumer Key
consumer_key = "Qi2d0rRBRzO3nrUcMSuSAvx1L"

In [4]:
# Adicione aqui sua Consumer Secret 
consumer_secret = "EkTp2iUAyztzDJBsgeVBTq5oZH8DJL57rgxD1Al0eWE6GPqNsk"

In [5]:
# Adicione aqui seu Access Token
access_token = "114524288-LnbGyP9BWln8vzqXbEtkzVOnBWX4wXEI9J9MoVtJ"

In [6]:
# Adicione aqui seu Access Token Secret
access_token_secret = "3L4gXJbYqKX6wEsUYXObQzqrWhd21VhocnL2Stu0yptAR"

In [7]:
# Criando as chaves de autenticação
auth = OAuthHandler(consumer_key, consumer_secret)

In [8]:
auth.set_access_token(access_token, access_token_secret)

In [9]:
# Criando uma classe para capturar os stream de dados do Twitter e 
# armazenar no MongoDB
class MyListener(StreamListener):
    def on_data(self, dados):
        tweet = json.loads(dados)
        created_at = tweet["created_at"]
        id_str = tweet["id_str"]
        text = tweet["text"]
        obj = {"created_at":created_at,"id_str":id_str,"text":text,}
        tweetind = col.insert_one(obj).inserted_id
        print (obj)
        return True

In [10]:
# Criando o objeto mylistener
mylistener = MyListener()

In [11]:
# Criando o objeto mystream
mystream = Stream(auth, listener = mylistener)

## Preparando a Conexão com o MongoDB

In [2]:
# Importando do PyMongo o módulo MongoClient
from pymongo import MongoClient   

In [3]:
# Criando a conexão ao MongoDB
client = MongoClient('localhost', 27017)

In [4]:
# Criando o banco de dados twitterdb
db = client.twitterdb

In [5]:
# Criando a collection "col"
col = db.tweets 

In [16]:
# Criando uma lista de palavras chave para buscar nos Tweets
keywords = ['Big Data', 'Python', 'Data Mining', 'Data Science']

## Coletando os Tweets

In [17]:
# Iniciando o filtro e gravando os tweets no MongoDB
mystream.filter(track=keywords)

{'created_at': 'Sat Sep 15 15:21:04 +0000 2018', 'id_str': '1040983847574224896', 'text': 'Senior Data Scientist: New York City, NY Full-time $100k – $160k Posted 08/10/2018 Location: Remote with 25% travel… https://t.co/cAGs4jFN1d', '_id': ObjectId('5b9d23606932d50e540989f8')}
{'created_at': 'Sat Sep 15 15:21:04 +0000 2018', 'id_str': '1040983849763782664', 'text': 'RT @BigDataGal: How To Build A Recommendation Engine in R | A Marketing Data Science Demo https://t.co/kz8FxOI5Hs #ai #iot #bigdata #automa…', '_id': ObjectId('5b9d23606932d50e540989f9')}
{'created_at': 'Sat Sep 15 15:21:07 +0000 2018', 'id_str': '1040983858441678848', 'text': 'RT @dina_ardiyanti: Revolusi 4.0 harus disikapi dengan gerakan inovasi yg berjejaring di desa dan kota, di sisi lain negara juga dituntut u…', '_id': ObjectId('5b9d23636932d50e540989fa')}
{'created_at': 'Sat Sep 15 15:21:07 +0000 2018', 'id_str': '1040983859511218177', 'text': 'Webinar: How Google BigQuery and Looker Can Accelerate Your Data Science

KeyboardInterrupt: 

## --> Pressione o botão Stop na barra de ferramentas para encerrar a captura dos Tweets

## Consultando os Dados no MongoDB

In [18]:
mystream.disconnect()

In [6]:
# Verificando um documento no collection
col.find_one()

{'_id': ObjectId('5b9d23606932d50e540989f8'),
 'created_at': 'Sat Sep 15 15:21:04 +0000 2018',
 'id_str': '1040983847574224896',
 'text': 'Senior Data Scientist: New York City, NY Full-time $100k – $160k Posted 08/10/2018 Location: Remote with 25% travel… https://t.co/cAGs4jFN1d'}

## Análise de Dados com Pandas e Scikit-Learn

In [7]:
# criando um dataset com dados retornados do MongoDB
dataset = [{"created_at": item["created_at"], "text": item["text"],} for item in col.find()]

In [8]:
# Importando o módulo Pandas para trabalhar com datasets em Python
import pandas as pd

In [9]:
# Criando um dataframe a partir do dataset 
df = pd.DataFrame(dataset)

In [10]:
# Imprimindo o dataframe
df

,created_at,text
0,Sat Sep 15 15:21:04 +0000 2018,"Senior Data Scientist: New York City, NY Full-..."
1,Sat Sep 15 15:21:04 +0000 2018,RT @BigDataGal: How To Build A Recommendation ...
2,Sat Sep 15 15:21:07 +0000 2018,RT @dina_ardiyanti: Revolusi 4.0 harus disikap...
3,Sat Sep 15 15:21:07 +0000 2018,Webinar: How Google BigQuery and Looker Can Ac...
4,Sat Sep 15 15:21:08 +0000 2018,Beautiful Monty Python ...
5,Sat Sep 15 15:21:13 +0000 2018,RT @profcarroll: Says the guy who’s (former) d...
6,Sat Sep 15 15:21:16 +0000 2018,RT @BenGrewell: Video on @hackthebox_eu system...


In [11]:
# Importando o módulo Scikit Learn
from sklearn.feature_extraction.text import CountVectorizer

In [24]:
# Usando o método CountVectorizer para criar uma matriz de documentos
cv = CountVectorizer()
count_matrix = cv.fit_transform(df.text)

In [25]:
# Contando o número de ocorrências das principais palavras em nosso dataset
word_count = pd.DataFrame(cv.get_feature_names(), columns=["word"])
word_count["count"] = count_matrix.sum(axis=0).tolist()[0]
word_count = word_count.sort_values("count", ascending=False).reset_index(drop=True)
word_count[:50]

,word,count
0,rt,29
1,co,20
2,https,20
3,python,14
4,the,13
5,yontengop,13
6,to,11
7,ぼく,10
8,上司さんメール,10
9,and,7


# Fim

### Obrigado - Data Science Academy - <a href="http://facebook.com/dsacademybr">facebook.com/dsacademybr</a>